# Examen
Ing. Angel Lopez Manriquez

En este cuaderno se muestran las primeras ideas que tuve al ver las reglas de negocio con algunas cosas que se asumieron.

Se creo una base de datos en MySQL con dos relaciones, *actor(id: int, name: str)* de la segunda hoja del excel, donde se asume que el indice dado corresponde a la llave primaria y no tiene cambios en el excel. Otra relacion *actor_relation(id: int, actor_id: int)* la cual guarda unicamente los valores *(i, j)* que indica que *i* esta relacionado con *j* (se asumio que la matriz representa un DAG).

En el codigo no se guarda un historico de los valores previos, solo se actualizan los valores, aunque en dagster si que se guardan logs de las operaciones realizadas. Si se hubiera querido guardar los valores se pudieron haber creado fragmentos en la base (creacion dinamica de relaciones) con la fecha de corte o la anidacion del cambio de fecha en las tablas.

Se pudo haber usado Spark, sin embargo el volumen de datos no lo ameritaba.

In [1]:
import pandas as pd
import os
import pymysql


from collections import defaultdict
from scipy.sparse import csr_matrix


In [119]:
def load_excel(fname: str) -> pd.ExcelFile:
    '''Se lee el excel.'''
    # return pd.ExcelFile(os.path.join('actors', 'resources', fname))
    return pd.ExcelFile(os.path.join('resources', fname))


fname = 'Matriz_de_adyacencia_data_team.xlsx'
xls = load_excel(fname)

In [120]:
def get_actors(xls: pd.ExcelFile) -> list:
    '''Se obtienen los actores.'''
    df = pd.read_excel(xls, 'Lista de actores', header=3, usecols="C")
    return df.values.squeeze().tolist()


actors = get_actors(xls)

In [131]:


def get_matrix_from_excel(xls: pd.ExcelFile):
    '''
    se carga la matrix dispersa o bien matriz de adyacencia, se asume que
    se representa un DAG
    '''
    df = pd.read_excel(xls, sheet_name='Matriz de adyacencia')
    return df.iloc[1:, 2:].values


m = len(actors)
matrix = get_matrix_from_excel(xls)
assert matrix.shape == (m, m)  # pytest, se espera una matriz cuadrada
matrix[:5]  

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]], dtype=object)

In [134]:

def dense_matrix_as_defaultdict(mat) -> defaultdict:
    ''' Se cargan los valores que se ocupan, (i, j) -> 1 donde i se relaciona con j
    '''
    result = defaultdict(set)
    m = mat.shape[0]
    for i in range(m):
        for j in range(i, m):  # empieza en i para no rebuscar
            if mat[i][j] == 1:  # ignora 0, np.nan
                result[i + 1].add(j + 1)
    return result


new_table = dense_matrix_as_defaultdict(matrix)

# Mysql
A partir de aqui, se hacen consultas con el controlador para MySQL.


In [7]:
conn = pymysql.connect(
        host="localhost", user="root", password="root", database="actors", port=3306)
cursor = conn.cursor()

In [6]:
conn.port

3306

In [124]:
def load_actors(conn, actors) -> None:
    '''Se cargan los actores dada una lista de actores List[Tuple[int, str]]. 
    Se asume que la hoja de los actores con su nombre no cambia por lo que la funcionalidad 
    de esta funcion en teoria solo corre una vez.'''
    cursor = conn.cursor()
    conn.begin()
    try:
        for i, name in enumerate(actors):
            cursor.callproc('usp_actor_add', (i + 1, name))
        conn.commit()
    except Exception as e:
        print("Ya se cargaron previamente los actores (%s,%s)" % (i + 1, name))
        print(e)


load_actors(conn, actors)

Ya se cargaron previamente los actores (0,Helena Durán Crane)
(1062, "Duplicate entry '1' for key 'actor.PRIMARY'")


In [106]:
def write(cursor, query, args=None):
    if args:
        cursor.execute(query, args)
    else:
        cursor.execute(query)

def read(cursor, query):
    cursor.execute(query)
    return cursor.fetchall()


In [135]:

def get_old_table(cursor) -> defaultdict:
    '''Cargamos los valores (i, j) de la relacion actual.'''
    result = read(cursor, "SELECT id, actor_id FROM actor_relation")
    relations = defaultdict(set)
    for i, j in result:
        relations[i].add(j)
    return relations


old_table = get_old_table(cursor)

In [127]:
import random


def get_random_table(a, b) -> defaultdict:
    '''Generamos un defaultdict aleatorio dentro de los rangos de la matriz creada con el fin 
    de probar nuestra funcionalidad.'''
    new_table = defaultdict(set)
    for i in range(a, b + 1):
        for j in range(6):
            new_table[i].add(random.randint(a, b))
    return new_table

a, b = 1, 50
new_table = get_random_table(1, 50)

Se muestran las supuestas tablas, la relacion actual y la relacion por ser reemplazada.

In [136]:
i = 13
print(old_table[i])
print(new_table[i])

{3, 38, 8, 14, 20}
{45, 14, 19, 22, 23, 24, 25, 26}


Este es el ejemplo de una fila, donde se muestran los valores a borrar y los nuevos valores a insertar. Los valores sin cambio permanecen.

In [137]:
a, b = old_table[i], new_table[i]
deprecated = a - b
new_relations = b - a
print(i)
deprecated, new_relations

13


({3, 8, 20, 38}, {19, 22, 23, 24, 25, 26, 45})

esta funcion hace lo anterior para todas

In [132]:
def update_relations(conn, old_table: defaultdict, new_table: defaultdict(set)) -> None:
    conn.begin()
    cursor = conn.cursor()
    idx = set(old_table.keys()) | set(new_table.keys())  # barre sobre cualquier cambio significativo en fila
    for i in idx:
        a, b = old_table[i], new_table[i]
        old_row = a - b  
        new_row = b - a
        for j in old_row:
            write(cursor, "DELETE FROM actor_relation WHERE id = %s AND actor_id = %s", (i, j))
        for j in new_row:
            cursor.callproc('usp_actor_relation_add', (i, j))
    conn.commit()
    

In [138]:
%time update_relations(conn, old_table, new_table)

CPU times: user 23.9 ms, sys: 8.19 ms, total: 32.1 ms
Wall time: 818 ms
